In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline
plt.viridis()
import time
from brian2.units import *
from scipy.spatial.distance import minkowski

In [4]:
def distance(s, t, grid_shape, dimensions):
        '''
        Function that computes distance in a grid of neurons taking into account periodic boundry conditions.

        First, translate source into the center of the grid.
        Second, translate target by the same amount.
        Finally, perform desired distance computation.
        '''
        s = np.asarray(s)
        t = np.asarray(t)
        _grid_size = np.asarray(grid_shape)
        trans = s - (_grid_size // 2)
        s = np.mod(s - trans, _grid_size)
        t = np.mod(t - trans, _grid_size)
        return minkowski(s, t, dimensions)

In [11]:
def generate_rates(s, grid_shape, dimensions):
    '''
    Function that generates an array the same shape as the input layer so that 
    each cell has a value corresponding to the firing rate for the neuron
    at that position.
    '''
    _rates = np.zeros(grid_shape)
    for x, y in np.ndindex(grid_shape):
        _d = distance(s, (x,y), grid_shape=grid_shape, dimensions=dimensions)
        _rates[x, y] = f_base + f_peak * np.e ** (-_d/(2 * sigma_stim**2))
    return _rates * Hz

In [20]:
duration = 100 * ms
d = 2
# Wiring
n = 16
N_layer = n ** 2
S = (n, n)

s = (n//2, n//2)
sigma_form_forward = 2.5
sigma_form_lateral = 1

# Inputs
f_mean = 20 * Hz
f_base = 5 * Hz
f_peak = 152.8  * Hz
sigma_stim = 2
t_stim = 0.02 * second

In [12]:
_chunk = 20 * ms
_dt = .1 * ms
_chunk / _dt

200.0

In [13]:
smth = [[],]

In [15]:
smth.append([3,2,1])

In [16]:
smth

[[], [3, 2, 1]]

The idea is to generate a 2D spike time array (different times for different spike source). To obtain a set a times I need to consider the rate of each spiking source, then generate a poisson series, and convert nonzero indices to times.

In [60]:
def generate_spike_times(s, dt=.1 * ms, chunk = 20 * ms, time_offset=0 * second, grid_shape=(16,16), dimensions=2):
    rates = generate_rates(s, grid_shape, dimensions)
    spike_times = []
    for rate in rates.ravel():
        spikes = np.random.poisson(rate/Hz/1000., int(chunk/dt))
        spike_times.append(((np.nonzero(spikes)[0] * dt + time_offset)/ms).tolist())
    return spike_times

In [61]:
spike_times = generate_spike_times((13,8), grid_shape=S, dimensions=2)

In [62]:
spike_times

[[0.1, 0.2, 0.6000000000000001, 2.8, 7.3, 9.2, 19.599999999999998],
 [4.0,
  4.8999999999999995,
  5.7,
  5.800000000000001,
  6.1000000000000005,
  7.7,
  8.1,
  11.799999999999999,
  11.9,
  15.3,
  18.0,
  18.8,
  19.200000000000003],
 [1.7000000000000002,
  2.3,
  3.6,
  3.9000000000000004,
  4.0,
  4.2,
  8.0,
  8.1,
  9.9,
  10.5,
  11.2],
 [0.1,
  2.1,
  2.3,
  3.6,
  4.7,
  6.2,
  6.800000000000001,
  8.1,
  10.0,
  11.9,
  15.5,
  15.600000000000001,
  16.400000000000002,
  17.0,
  18.0,
  18.3,
  19.0],
 [2.6,
  3.3,
  3.7,
  5.1000000000000005,
  6.0,
  6.1000000000000005,
  7.5,
  7.800000000000001,
  8.0,
  13.2,
  13.8,
  14.700000000000001,
  16.099999999999998,
  16.2,
  17.3],
 [0.1,
  0.2,
  1.6,
  3.6,
  4.6,
  5.2,
  5.7,
  7.2,
  7.4,
  7.6,
  7.7,
  8.4,
  10.8,
  11.5,
  12.8,
  14.200000000000001,
  14.4,
  14.5,
  16.099999999999998,
  18.500000000000004,
  19.4,
  19.5,
  19.700000000000003,
  19.900000000000002],
 [3.3,
  4.7,
  5.3,
  6.3,
  6.9,
  7.0,
  7.